In [ ]:
import gdown
import tarfile
import xml.etree.ElementTree as ET
import os
import xml.dom.minidom
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import seaborn as sns
import numpy as np
import regex as re
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from PIL import Image
import tensorflow as tf
import cv2 as cv
from keras.models import load_model
from tensorflow.keras.applications import DenseNet121
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.backend import expand_dims
from tensorflow import concat
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.layers import TimeDistributed
tf.keras.backend.clear_session()
from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM,Layer,Dropout
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import repeat
from sklearn.utils import shuffle
import nltk.translate.bleu_score as bleu

In [ ]:
url = 'https://drive.google.com/uc?id=1U7D9lnjH-0CaXzhmua2rMtmdgS4DEH2k'
output = 'proccessed.pkl'
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1U7D9lnjH-0CaXzhmua2rMtmdgS4DEH2k
To: /content/proccessed.pkl
100%|██████████| 58.1M/58.1M [00:00<00:00, 82.2MB/s]


'proccessed.pkl'

In [ ]:
data = pd.read_pickle('/content/proccessed.pkl')


In [ ]:
data.head()


,patient_id,image1,image2,findings,image_features,findings_total,dec_ip,dec_op
0,CXR3556,/content/png/CXR3556_IM-1741-1001-0001.png,/content/png/CXR3556_IM-1741-1001-0002.png,the lungs are clear. there is no pleural effus...,"[[0.00041582860285416245, 0.001570420921780169...",<start> the lungs are clear. there is no pleur...,<start> the lungs are clear. there is no pleur...,the lungs are clear. there is no pleural effus...
2,CXR32,/content/png/CXR32_IM-1511-1001.png,/content/png/CXR32_IM-1511-4001.png,the heart is normal in size. the mediastinum i...,"[[0.0005354544264264405, 0.0019668142776936293...",<start> the heart is normal in size. the media...,<start> the heart is normal in size. the media...,the heart is normal in size. the mediastinum i...
4,CXR260,/content/png/CXR260_IM-1090-1001.png,/content/png/CXR260_IM-1090-2001.png,lungs are clear bilaterally. cardiac and media...,"[[0.0002745636156760156, 0.0018877924885600805...",<start> lungs are clear bilaterally. cardiac a...,<start> lungs are clear bilaterally. cardiac a...,lungs are clear bilaterally. cardiac and media...
5,CXR1301,/content/png/CXR1301_IM-0198-1001.png,/content/png/CXR1301_IM-0198-2001.png,"heart size within normal limits, stable medias...","[[0.0005874697235412896, 0.0018448150949552655...","<start> heart size within normal limits, stabl...","<start> heart size within normal limits, stabl...","heart size within normal limits, stable medias..."
6,CXR1921,/content/png/CXR1921_IM-0598-1001.png,/content/png/CXR1921_IM-0598-2001.png,"redemonstration of moderately-inflated lungs, ...","[[0.00029747304506599903, 0.0014215346891433, ...",<start> redemonstration of moderately-inflated...,<start> redemonstration of moderately-inflated...,"redemonstration of moderately-inflated lungs, ..."


In [ ]:
X_train, X_test = train_test_split(data.values , test_size = 0.2 )


In [ ]:
X_train.shape


(2669, 8)

In [ ]:
X_test.shape


(668, 8)

In [ ]:
X_train = X_train[:-19, :]
X_test = X_test[:-18, :]


In [ ]:
t1 = Tokenizer( filters='!"#$%&()*+,-/:;=?@[\\]^_`{|}~\t\n',oov_token='OOV')
t1.fit_on_texts(X_train[:,5])

t1.word_index['<pad>'] = 0
t1.index_word[0] = '<pad>'
vocab_size_imp = len(t1.word_index) + 1

dec_inp = t1.texts_to_sequences(X_train[:,6])

dec_inp = pad_sequences(dec_inp, maxlen=98, padding='post')

dec_inp_cv = t1.texts_to_sequences(X_test[:,6])

dec_inp_cv = pad_sequences(dec_inp_cv, maxlen=98, padding='post')

dec_op = t1.texts_to_sequences(X_train[:,7])

dec_op = pad_sequences(dec_op, maxlen=98, padding='post')

dec_op_cv = t1.texts_to_sequences(X_test[:,7])

dec_op_cv = pad_sequences(dec_op_cv, maxlen=98, padding='post')


In [ ]:
vocab_size_imp


1826

In [ ]:
imp1 = {}
imp2 = {}
for key,value in t1.word_index.items():
  imp1[value] = key
  imp2[key] = value


In [ ]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''

    def __init__(self):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.dense1 = Dense(512)
        self.d1 = Dropout(0.5)


    def call(self,image_data):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        self.image_data = image_data
        self.enc_out = self.dense1(self.image_data)
        self.enc_out =self.d1(self.enc_out )


        return self.enc_out





In [ ]:
class Decoder(Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,out_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Decoder LSTM layer
        super().__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size = lstm_size
        self.d2  = Dropout(0.3)
        self.dec_emb = Embedding(out_vocab_size,300,trainable = True)            #(None , 12,embedding_size)
        self.dec_lstm = LSTM(self.lstm_size, return_sequences=True, name="Decoder_LSTM")  #(None , 12,lstm_size)




    def call(self,input_sequence):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to decoder_lstm

          returns -- decoder_output,decoder_final_state_h,decoder_final_state_c
        '''
        #print("DECODER ==> INPUT SQUENCES SHAPE :",target_sentances.shape)
        self.input_sequence = input_sequence

        self.target_embedd           = self.dec_emb (self.input_sequence)
        self.target_embedd =self.d2(self.target_embedd )
        #print("WE ARE INITIALIZING DECODER WITH ENCODER STATES :",state_h.shape, state_c.shape)
        lstm_output      = self.dec_lstm(self.target_embedd)

        return lstm_output



In [ ]:
class Encoder_decoder(Model):

    #def __init__(self,*params):
    def __init__(self,out_vocab_size,embedding_size_d,lstm_size_d,input_length_d,batch_size):

        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
        super().__init__()


        self.out_vocab_size = out_vocab_size
        self.embedding_size_d = embedding_size_d
        self.lstm_size_d = lstm_size_d
        self.input_length_d = input_length_d
        self.batch_size = batch_size

        self.encoder = Encoder()

        self.decoder = Decoder(out_vocab_size , embedding_size_d, lstm_size_d,input_length_d )
        self.dense   = TimeDistributed(Dense(self.out_vocab_size, activation='softmax'))
        self.d3 = Dropout(0.3)


    #def call(self,*params):
    def call(self,data):

        '''
        A. Pass the input sequence to Encoder layer -- Return encoder_output,encoder_final_state_h,encoder_final_state_c
        B. Pass the target sequence to Decoder layer with intial states as encoder_final_state_h,encoder_final_state_C
        C. Pass the decoder_outputs into Dense layer

        Return decoder_outputs
        '''
        self.input1,self.input2 = data[0], data[1]
        print("="*20, "ENCODER", "="*20)
        self.encoder_output = self.encoder(self.input1)
        print("-"*27)
        #print("ENCODER ==> OUTPUT SHAPE",self.encoder_output.shape)
        #print("ENCODER ==> HIDDEN STATE SHAPE",self.encoder_h.shape)
        #print("ENCODER ==> CELL STATE SHAPE", self.encoder_c.shape)
        print("="*20, "DECODER", "="*20)
        self.decoder_output  = self.decoder(self.input2)
        #self.decoder_output =Dropout(0.3)(self.decoder_output)
        self.add=tf.keras.layers.Add()([self.encoder_output, self.decoder_output])
        self.add =self.d3(self.add)

        output         = self.dense(self.add)
        print("-"*27)
        print("FINAL OUTPUT SHAPE",output.shape)
        print("="*50)
        return output




In [ ]:
#Create an object of encoder_decoder Model class,
# Compile the model and fit the model
model1 = Encoder_decoder(vocab_size_imp,300,512,36,50)


In [ ]:
!mkdir checkpoint


In [ ]:
cwd = os.getcwd()


In [ ]:
checkpoint_filepath = cwd + '/' + 'checkpoint' + '/'


In [ ]:
checkpoint_filepath


'/content/checkpoint/'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    verbose = 1,
    mode='min',
    save_best_only=True)


In [ ]:
tf.keras.backend.clear_session()


In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8,mode = 'min',verbose = 1,
                              patience=1, min_lr=0.0001)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
model1.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy', metrics = 'accuracy')


In [ ]:
train_inp = np.vstack(X_train[:,4]).astype(float)
test_inp = np.vstack(X_test[:,4]).astype(float)


In [ ]:
model1.fit([train_inp,dec_inp ],dec_op ,validation_data= ([test_inp, dec_inp_cv],dec_op_cv),batch_size= 25,epochs  = 30,callbacks=[reduce_lr,model_checkpoint_callback])


Epoch 1/30
==================== ENCODER ====================
---------------------------
==================== DECODER ====================
---------------------------
FINAL OUTPUT SHAPE (25, 98, 1826)
==================== ENCODER ====================
---------------------------
==================== DECODER ====================
---------------------------
FINAL OUTPUT SHAPE (25, 98, 1826)
106/106 [==============================] - ETA: 0s - loss: 1.9919 - accuracy: 0.6744==================== ENCODER ====================
---------------------------
==================== DECODER ====================
---------------------------
FINAL OUTPUT SHAPE (25, 98, 1826)

Epoch 1: val_loss improved from inf to 1.63379, saving model to /content/checkpoint/
106/106 [==============================] - 24s 149ms/step - loss: 1.9919 - accuracy: 0.6744 - val_loss: 1.6338 - val_accuracy: 0.6947 - lr: 0.0010
Epoch 2/30
106/106 [==============================] - ETA: 0s - loss: 1.5825 - accuracy: 0.7011
Epoch 

In [ ]:
def predict(input_model):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to decoder
  D. till we reach max_length of decoder or till the model1 predicted word <end>:
         predicted_out,state_h,state_c=model1.layers[1](dec_input,states)
         pass the predicted_out to the dense layer
         update the states=[state_h,state_c]
         And get the index of the word with maximum probability of the dense layer output, using the tokenizer(word index) get the word and then store it in a string.
         Update the input_to_decoder with current predictions
  F. Return the predicted sentence
  '''
  input_sentence = input_model

  encoder_output= model1.layers[0](input_sentence)#,self.initial_state)
  encoder_output = model1.layers[0].d1(encoder_output)

  cur_vec = np.ones((1, 1)) * imp2['<start>'] # Here replace with index of <start> in the english vocab
  pred = []
  predicted_senence = "<start>"
  for i in range(20):
    cur_emb = model1.layers[1].dec_emb(cur_vec)
    cur_emb = model1.layers[1].d2(cur_emb)
    infe_output= model1.layers[1].dec_lstm(cur_emb)
    infe_output=tf.keras.layers.Add()([encoder_output, infe_output])
    infe_output=model1.layers[2](infe_output)
    cur_vec = np.reshape(np.argmax(infe_output), (1, 1))
    pred.append(cur_vec[0][0])

    predicted_senence = predicted_senence + ' ' + imp1[cur_vec[0][0]+1]
    if(cur_vec[0][0] == imp2['<end>'] ):
      break
  return predicted_senence.strip()

#max_length = int(padlength)
#max_length


In [ ]:
num = 36
im_o = np.vstack(X_test[:,4][num]).astype(float)
tex_o = X_test[:,5][num]
print("Original Sentence is : ",tex_o)
print("Predicted sentence: ",predict(im_o))


Original Sentence is :  <start> clear lungs. normal heart. no pneumothora. no pleural effusion. old right rib fractures. <end>
Predicted sentence:  <start> no focal is with appear size clear or pulmonary of or pulmonary of or pulmonary of or pulmonary of or


In [ ]:
num = 90
im_o = np.vstack(X_test[:,4][num]).astype(float)
tex_o = X_test[:,5][num]
print("Original Sentence is : ",tex_o)
print("Predicted sentence: ",predict(im_o))


Original Sentence is :  <start> the heart is normal in size. the mediastinum is unremarkable. there is slight elevation of left hemidiaphragm with left basilar subsegmental atelectasis. the lungs are otherwise grossly clear. <end>
Predicted sentence:  <start> no left clear. <start> opacities. in heart lungs within pleural vasculature noted. <start> opacities. in heart lungs within pleural vasculature


In [ ]:
num = 102
im_o = np.vstack(X_test[:,4][num]).astype(float)
tex_o = X_test[:,5][num]
print("Original Sentence is : ",tex_o)
print("Predicted sentence: ",predict(im_o))


Original Sentence is :  <start> the cardiomediastinal silhouette is within normal limits for size and contour. the lungs are normally inflated without evidence of focal airspace disease, pleural effusion, or pneumothora. osseous structures are within normal limits for patient age. <end>
Predicted sentence:  <start> no focal is airspace normal <start> opacities. lung chest. from no focal is airspace normal <start> opacities. lung chest. from


In [ ]:
b = 0


for i in range(X_test.shape[0]):
  im_o = np.vstack(X_test[:,4][i]).astype(float)
  pred = predict(im_o)
  org= X_test[:,5][i]

  b=  b + bleu.sentence_bleu([org.split()], pred.split() )

print("Bleu score is : ",b/X_test.shape[0])


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Bleu score is :  1.4425136149869226e-80
